In [1]:
%run ../src/ipyautoui/__init__.py
%load_ext lab_black

# AutoUi

With ipyautoui we can create ipywidgets from either a **json-schema** or a **pydantic model**. This makes it quick and easy to whip up a user interface when required.

In [2]:
from ipyautoui import demo

demo()

AttributeError: 'NoneType' object has no attribute 'items'

In [8]:
w.Dropdown(options=["a", "b"], value=None)

Dropdown(options=('a', 'b'), value=None)

In [3]:
from ipyautoui import AutoUi
import json
from pydantic import BaseModel, Field
from ipyautoui.constants import DIR_MODULE
from ipyautoui._utils import display_pydantic_json
import ipyautoui
import ipywidgets as w

### Creating Simple Widget

In [4]:
from ipyautoui.test_schema import (
    TestAutoLogic,
    TestAutoLogicSimple,
)  # the schema shown in the file above

So let's create a simple pydantic class. Here we have one text field.

In [5]:
# create a pydantic model (or a json-schema) defining the fields of interest
class AutoUiExample(BaseModel):
    text: str = Field(default="Test", description="This description is very important")


data = {"text": "this is a value"}
ui = AutoUi(schema=AutoUiExample, path="test.ui.json")
display(ui)

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

In [39]:
# from ipyautoui.basemodel import file

file(AutoUiExample(), pathlib.Path("test.json"))

In [30]:
"""extending default pydantic BaseModel. NOT IN USE."""
import pathlib
from pydantic import BaseModel
from typing import Type


def file(self: Type[BaseModel], path: pathlib.Path, **json_kwargs):
    """
    this is a method that is added to the pydantic BaseModel within AutoUi using
    "setattr".

    Example:
        ```setattr(model, 'file', file)```

    Args:
        self (pydantic.BaseModel): instance
        path (pathlib.Path): to write file to
    """
    if "indent" not in json_kwargs.keys():
        json_kwargs.update({"indent": 4})
    path.write_text(self.json(**json_kwargs), encoding="utf-8")


aui = AutoUiExample()
aui.__slots__ = ('file',)
setattr(aui, "file", file)


#object.__setattr__(aui, "file", file)
aui.file(pathlib.Path('test.json'))

ValueError: "AutoUiExample" object has no field "__slots__"

AttributeError: 'AutoUiExample' object has no attribute 'file'

### Writing to JSON

Let's define the save location.

In [7]:
import pathlib

save_path = pathlib.Path(".") / "test.simpleaui.json"
print(f"Save Location is: {save_path}")

Save Location is: test.simpleaui.json


In [12]:
ui.file(path=save_path)
AutoUiRenderer = AutoUi.create_autoui_renderer(schema=AutoUiExample)

ui_simple = AutoUiRenderer(path=save_path)


def test_action():
    print("done")


ui_simple.savebuttonbar.fns_onsave_add_action(test_action)
ui_simple.show_savebuttonbar = True
display(ui_simple)

AutoRenderer(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True…

In [9]:
import ipywidgets as w

# ui_simple.autowidget.insert_rows = {0: w.Button()}

In [10]:
import typing as ty


class DataFrameCols(BaseModel):
    string: str = Field("string", aui_column_width=100)
    integer: int = Field(1, aui_column_width=80)
    floater: float = Field(3.1415, aui_column_width=70, aui_sig_fig=3)
    something_else: float = Field(324, aui_column_width=100)


class TestDataFrame(BaseModel):
    """a description of TestDataFrame"""

    dataframe: ty.List[DataFrameCols] = Field(
        default_factory=lambda: [], format="dataframe"
    )

In [11]:
auto_grid = AutoUi(schema=TestDataFrame)
display(auto_grid)

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

In [13]:
auto_grid.value = {
    "dataframe": [
        {
            "string": "important string",
            "integer": 1,
            "floater": 3.14,
            "something_else": 324,
        },
        {"string": "update", "integer": 4, "floater": 3.12344, "something_else": 123},
        {"string": "evening", "integer": 5, "floater": 3.14, "something_else": 235},
        {"string": "morning", "integer": 5, "floater": 3.14, "something_else": 12},
        {"string": "number", "integer": 3, "floater": 3.14, "something_else": 123},
    ]
}